In [186]:
!pip install bs4
!pip install urllib3
!pip install selenium

In [189]:
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np
import re
import selenium
from urllib.request import Request, urlopen
from urllib import request
import requests
import time
from functools import reduce
import sys 
import webbrowser
import requests 
import csv
from tqdm import tqdm_notebook as tqdm

#### Création des listes de liens

In [191]:

liste_pages_tgpreop = []
for i in range(300) : 
  o = i+1
  liste_pages_tgpreop.append('https://www.theeroticreview.com'  + f'/reviews/newreviewsList.asp?Valid=1&mp=0&SortBy=3&searchreview=1&Transsexual=2&gDistance=0&page={o}')
#liste_pages

In [192]:
liste_pages_tgpostop = []
for i in range(14) : 
  o = i+1
  liste_pages_tgpostop.append('https://www.theeroticreview.com'  + f'/reviews/newreviewsList.asp?searchreview=1&Transsexual=3&SortBy=3&gDistance&page={o}')
#liste_pages

In [193]:
liste_pages_cs = []
for i in range(9000) : 
  o = i+1
  liste_pages_cs.append('https://www.theeroticreview.com'  + f'/reviews/newreviewsList.asp?searchreview=1&Transsexual=1&SortBy=3&gDistance=0&page={o}')
#liste_pages_cs

In [194]:
liste_pages=  liste_pages_tgpostop+  liste_pages_tgpreop #+liste_pages_cs

In [195]:
page = {}
request_text = {}
tableau_participants = {}
links = []
for i in range(len(liste_pages)) :
  request_text[i] = urllib.request.urlopen(liste_pages[i]).read()
  page[i] = BeautifulSoup(request_text[i], "lxml")
  tableau_participants[i] = page[i].find('div',{'class' : "ter-table"})
  #print(tableau_participants[i])
  
  for link in tableau_participants[i].findAll('a'):
   links.append(link.get("href"))

In [196]:
link=[]
for i in range(len(links)):
    link.append('https://www.theeroticreview.com' + links[i])

In [197]:
link=pd.DataFrame({'1':link})
link.to_csv('link.csv', index=False)

In [198]:
link=[]
for i in range(len(links)):
    link.append('https://www.theeroticreview.com' + links[i])

#### Définition des fonctions de créations de DataFrame

In [199]:
def my_funccol4(tableau_va4) :
  tbl = tableau_va4.find_all('div', {"class" : "profile-cost-container-reversed-td profile-cost-container-reversed-td-h col-xs-12"})
  tbl1 = []
  for i in range(len(tbl)) :
    tbl1.append(tbl[i].get_text())
  df4a = pd.DataFrame({'':tbl1})
  df4a = df4a.T
  df4a['ID'] = 'ID'

  tblb = tableau_va4.find_all('div', {"class" : "profile-cost-container-reversed-td col-xs-12"})
  tblb1 = []
  for i in range(len(tblb)) :
    tblb1.append(tblb[i].get_text())
  df4ab = pd.DataFrame({'':tblb1})
  df4ab = df4ab.T
  df4ab['ID'] = 1


  df = pd.concat([df4a, df4ab], ignore_index=False)

  new_header = df.iloc[0] #grab the first row for the header
  df = df[1:] #take the data less the header row
  df.columns = new_header 

  return(df)

In [200]:
def my_func(tableau_va1) : 
  tbl = tableau_va1.find_all("div",{"class":"profile-table-td profile-table-td-h col-xs-6"})
 ### Là je fais ce qu'on a déjà pu faire avant, j'ai selectionné qu'une partie
 ### du code HTML. Là c'est la partie "question" des informations qui sont présentes
 ### sur le site
  tbl1 = [] ### là on fait une petite liste vide 

  for i in range(len(tbl)) :
    tbl1.append(tbl[i].get_text()) ### la fondtion get_text récupère les string au
    ### milieu des balises et donc ça c'est vraiment super 
    ### à la fin dans tbl1 on a une liste qui contient toutes les informations qu'on veut

  
  df = pd.DataFrame({'':tbl1})
  df = df.T
  df['ID'] = 'ID'
  ### transformation de la liste en DataFrame + ajout de la colonne ID 
  
  tbla = tableau_va1.find_all("div",{"class":"profile-table-td col-xs-6"})
  tbl2 = []
  for i in range(len(tbl)) :
    tbl2.append(tbla[i].get_text())
  tbl2

  ff2 = pd.DataFrame({})
  df2 = pd.DataFrame({'1':tbl2})
  df2 = df2.T
  df2['ID'] = 1

  df = pd.concat([df, df2], ignore_index=False)

  new_header = df.iloc[0] #grab the first row for the header
  df = df[1:] #take the data less the header row
  df.columns = new_header 

  return(df)

In [201]:
a = ["collapse1", "collapse2", "collapse3"]
def my_2func(page):
  di = {}
  d = {}
  l_merge = []
  dfc4 = page.find('div', id = 'collapse4')
  dfc4 = my_funccol4(dfc4)
  l_merge.append(dfc4)
  for name in a:
    d[name] = page.find('div',id = name) ### là on a bien trois listes 
    ###d['collapse1'], d['collapse2'], d['collapse3']
    di[name] = my_func(d[name])
    l_merge.append(di[name])
  df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['ID'],
                                            how='outer'), l_merge)
  return(df_merged)

In [202]:
b = ["collapse4"]
def my_2funckk(page):
  di = {}
  d = {}
  l_merge = []
  for name in b :
    d[name] = page.find('div',id = name)
    di[name] = my_func(d[name])
    l_merge.append(di[name])
  df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['ID'],
                                            how='outer'), l_merge)
  return(df_merged)

#### Récupération des cookies et headers (à faire à chaque connexion)

In [210]:
cookies = {
    '_ga': 'GA1.2.1472492008.1636970512',
    'terAgreementVer': '1',
    'TERLocationResolved': '1',
    'TerTzOffset': '60',
    'TER%5FSearchGeoCityId': '',
    'TER%5FSearchGeoCityName': '',
    'cookieconsent_status': 'dismiss',
    'TER%5FSearchCountry': '',
    'TER%5FSearchCity': '',
    'language': 'en',
    'TER%5FRememberPW': '1',
    'TER%5FtestCookie': '',
    'TER%5Fusername': 'scrapptertable',
    'tz': 'Romance+Standard+Time',
    'MsTerVer': '6',
    'MPA3': '1',
    'K-GUID-pocpghie': '3F8CF11C54151A7BCB8706E92E026E87',
    '_gid': 'GA1.2.226639742.1644671558',
    'TER%5Fsplash': '1',
    'TER%5Fhash': 'gWHXPeR%2BG22m8LQbrP6pu3MNVG%2FmOsbQpaSLOJboF0GBwZNI0k%2FxeoyDakeCc8%2F8VZz436W7byfUA9fukLxZTA%3D%3D',
    'MsTerCounter': '3',
    'GUID': '80A890DE20A23041AE58F8133FB3EC5B',
    '_gat': '1',
}

headers = {
    'authority': 'www.theeroticreview.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
}


In [214]:
da = {}
frames = []
for i in tqdm(range(len(link))):
  URL = link[i]
  request_text = requests.get(URL, headers=headers, cookies=cookies)
  page = BeautifulSoup(request_text.text, 'lxml')
  da[i] = my_2func(page)
  da[i] = da[i].loc[:,~da[i].columns.duplicated()]
  #print(da[i])
  frames.append(da[i])
  #print(frames)
  df = pd.concat(frames, ignore_index=True)
df.to_csv('df.csv', index=False)

<ipython-input-214-cd1276c5f062>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(link))):


In [218]:
df.to_csv('df.csv',index=False)